# Pull Water Level Data for the Last 30 Days

## Imports and Functions

In [60]:
import pandas as pd
import requests
import json
from tqdm.auto import tqdm

API_STRING = 'https://api.mrcmekong.org/api/v1/nrtm/measurement/'

def get_df(station_code):
    with requests.get(f'{API_STRING}{station_code}') as r:
        j = json.loads(r.content)
        
    if len(j['measurements']) == 0:
        print(f'{station_code} returns no data.')
        return None
    
    df = pd.DataFrame(j['measurements'])
    
    df.columns = ['timestamp',
                  'water_level_m',
                  'rainfall_mm_15min',
                  'temperature_c',
                  'b']
    df.insert(1,'ymd', df.timestamp.map(lambda x: x[:10]))
    df.insert(2, 'hm', df.timestamp.map(lambda x: x[11:16]))
    df.insert(0, 'station_code', station_code)
    return df

## Get One Station

In [56]:
df = get_df(station_codes[0])
df

,station_code,timestamp,ymd,hm,water_level_m,rainfall_mm_15min,temperature_c,b
0,010501,2021-12-09T22:30:00+07:00,2021-12-09,22:30,2.130,0.0,NaN,NaN
1,010501,2021-12-09T22:45:00+07:00,2021-12-09,22:45,2.134,0.0,NaN,NaN
2,010501,2021-12-09T23:00:00+07:00,2021-12-09,23:00,2.139,0.0,NaN,NaN
3,010501,2021-12-09T23:15:00+07:00,2021-12-09,23:15,2.145,0.0,NaN,NaN
4,010501,2021-12-09T23:30:00+07:00,2021-12-09,23:30,2.158,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
2874,010501,2022-01-08T21:00:00+07:00,2022-01-08,21:00,1.759,0.0,NaN,NaN
2875,010501,2022-01-08T21:15:00+07:00,2022-01-08,21:15,1.759,0.0,NaN,NaN
2876,010501,2022-01-08T21:30:00+07:00,2022-01-08,21:30,1.761,0.0,NaN,NaN
2877,010501,2022-01-08T21:45:00+07:00,2022-01-08,21:45,1.764,0.0,NaN,NaN


## Get All Stations

In [61]:
station_codes = [
    '010501',
    '011903',
    '012001',
    '013101',
    '013402',
    '013801',
    '050106',
    '070103',
    '150101',
    '290102',
    '290113',
]

dfs = []
for station_code in tqdm(station_codes):
    d = get_df(station_code)
    dfs.append(d)

df = pd.concat(dfs).reset_index(drop=True)
df

  0%|          | 0/11 [00:00<?, ?it/s]

013101 returns no data.
290102 returns no data.


,station_code,timestamp,ymd,hm,water_level_m,rainfall_mm_15min,temperature_c,b
0,010501,2021-12-09T22:30:00+07:00,2021-12-09,22:30,2.130,0.0,NaN,NaN
1,010501,2021-12-09T22:45:00+07:00,2021-12-09,22:45,2.134,0.0,NaN,NaN
2,010501,2021-12-09T23:00:00+07:00,2021-12-09,23:00,2.139,0.0,NaN,NaN
3,010501,2021-12-09T23:15:00+07:00,2021-12-09,23:15,2.145,0.0,NaN,NaN
4,010501,2021-12-09T23:30:00+07:00,2021-12-09,23:30,2.158,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
25895,290113,2022-01-08T21:00:00+07:00,2022-01-08,21:00,1.893,0.0,None,NaN
25896,290113,2022-01-08T21:15:00+07:00,2022-01-08,21:15,1.893,0.0,None,NaN
25897,290113,2022-01-08T21:30:00+07:00,2022-01-08,21:30,1.893,0.0,None,NaN
25898,290113,2022-01-08T21:45:00+07:00,2022-01-08,21:45,1.893,0.0,None,NaN


In [69]:
#save temp
df.to_csv('data/df_temp.csv', index=False)

## Compare with Previous Day

In [83]:
df_previous = pd.read_csv(f'data/df_{df.ymd.unique()[-2]}.csv')
df_today = pd.concat([df,df_previous]).drop_duplicates()
df_today.groupby(['station_code','ymd']).hm.count()

station_code  ymd       
10501         2021-12-09     6
              2021-12-10    96
              2021-12-11    96
              2021-12-12    96
              2021-12-13    96
                            ..
290113        2022-01-04    96
              2022-01-05    96
              2022-01-06    96
              2022-01-07    96
              2022-01-08    89
Name: hm, Length: 558, dtype: int64

In [85]:
df_today.to_csv(f'data/df_{df.ymd.unique()[-1]}.csv',index=False)

In [86]:
df_today

,station_code,timestamp,ymd,hm,water_level_m,rainfall_mm_15min,temperature_c,b
0,010501,2021-12-09T22:30:00+07:00,2021-12-09,22:30,2.130,0.0,NaN,NaN
1,010501,2021-12-09T22:45:00+07:00,2021-12-09,22:45,2.134,0.0,NaN,NaN
2,010501,2021-12-09T23:00:00+07:00,2021-12-09,23:00,2.139,0.0,NaN,NaN
3,010501,2021-12-09T23:15:00+07:00,2021-12-09,23:15,2.145,0.0,NaN,NaN
4,010501,2021-12-09T23:30:00+07:00,2021-12-09,23:30,2.158,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
25895,290113,2022-01-08T21:00:00+07:00,2022-01-08,21:00,1.893,0.0,NaN,NaN
25896,290113,2022-01-08T21:15:00+07:00,2022-01-08,21:15,1.893,0.0,NaN,NaN
25897,290113,2022-01-08T21:30:00+07:00,2022-01-08,21:30,1.893,0.0,NaN,NaN
25898,290113,2022-01-08T21:45:00+07:00,2022-01-08,21:45,1.893,0.0,NaN,NaN
